In [141]:
import os
os.chdir('/freqtrade/')


import pandas as pd
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
from pathlib import Path

backtest_dir = config["user_data_dir"] / "backtest_results"

report_columns = [
    "Backtest",
    "Period",
    "Strategy",
    "MOT",
    "Total_Profit",
    "Max_Drawdown"
]
df = pd.DataFrame(columns = report_columns)

backtest_list = Path(backtest_dir).rglob('*.json')
for path in backtest_list:
    backtest_file_path = str(path)
        
    if ".last_result.json" in backtest_file_path:
        continue

    stats = load_backtest_stats(backtest_file_path)

    import ntpath
    backtest_name = ntpath.basename(backtest_file_path)
    
    for strategy, stats in stats['strategy'].items():
        
        period = stats['backtest_start'][0:10] + '__' + stats['backtest_end'][0:10]

        new_row = {
            "Backtest": '', #backtest_name,
            "Period": period,
            "Strategy": strategy,
            "MOT": stats['max_open_trades'],
            "Total_Profit": stats['profit_total'],
            "Max_Drawdown": stats['max_drawdown']
        }
        df = df.append(new_row, ignore_index=True)


print(df.groupby('Period').Total_Profit.transform(pd.Series.quantile, 0.85)[0])
print(df.groupby('Period').Max_Drawdown.transform(pd.Series.quantile, 0.15)[0])

filtered_df = df.loc[ 
    (df.Total_Profit >= df.groupby('Period').Total_Profit.transform(pd.Series.quantile, 0.85)) &
    (df.Max_Drawdown <= df.groupby('Period').Max_Drawdown.transform(pd.Series.quantile, 0.15))
]
print(" Filtered DF - Top Profit/Low Drawdown: ")
print(filtered_df.to_string())
print(" ********************** ")


print(" All backtests: ")
print(df.to_string())


698.0676933035528
0.3551453984999987
 Filtered DF - Top Profit/Low Drawdown: 
   Backtest                  Period                  Strategy MOT  Total_Profit  Max_Drawdown
22           2021-01-01__2021-05-27     CombinedBinHAndClucV7   1   2012.901495      0.314052
24           2021-01-01__2021-05-27  CombinedBinHClucAndMADV5   1   5749.340391      0.172414
 ********************** 
 All backtests: 
   Backtest                  Period                  Strategy MOT  Total_Profit  Max_Drawdown
0            2021-01-01__2021-05-27     CombinedBinHAndClucV6   2    321.227222      0.472416
1            2021-01-01__2021-05-27    CombinedBinHAndClucV6H   2    627.775796      0.500656
2            2021-01-01__2021-05-27     CombinedBinHAndClucV7   2    533.917267      0.452337
3            2021-01-01__2021-05-27     CombinedBinHAndClucV8   2    735.917176      0.729101
4            2021-01-01__2021-05-27  CombinedBinHClucAndMADV5   2    604.886380      0.235979
5            2021-01-01__2021-05-2